# Load model

In [3]:
model_monuseg_step1_prop01_path = r"D:\BaiduNetdiskDownload\Nudiff-checkpoints\diffusion\monuseg_step1\prop0.1\model130000.pt"
model_monuseg_step2_prop01_path = r"D:\BaiduNetdiskDownload\Nudiff-checkpoints\diffusion\monuseg_step2\prop0.1_finetune\model150000.pt"

In [4]:
import torch
import os

# Đường dẫn checkpoint
step1_ckpt = model_monuseg_step1_prop01_path
step2_ckpt = model_monuseg_step2_prop01_path

# Hàm load model (giả sử dùng torch và kiến trúc đã biết)
def load_model(ckpt_path, model_class, device='cuda' if torch.cuda.is_available() else 'cpu'):
    assert os.path.exists(ckpt_path), f"Checkpoint không tồn tại: {ckpt_path}"
    model = model_class()  # Thay bằng kiến trúc thực tế
    checkpoint = torch.load(ckpt_path, map_location=device)
    model.load_state_dict(checkpoint['model'] if 'model' in checkpoint else checkpoint)
    model.to(device)
    model.eval()
    print(f"✅ Loaded model from {ckpt_path}")
    return model

# TODO: Thay model_class bằng class thực tế của diffusion model
# from nudiff.struct_syn.unet import UNetModel
# from nudiff.image_syn.cond.unet_spade import UNetSPADE
# model1 = load_model(step1_ckpt, UNetModel)
# model2 = load_model(step2_ckpt, UNetSPADE)

# Nếu chưa có class, chỉ kiểm tra checkpoint
for ckpt in [step1_ckpt, step2_ckpt]:
    if os.path.exists(ckpt):
        print(f"Found checkpoint: {ckpt}")
        checkpoint = torch.load(ckpt, map_location='cpu')
        print(f"Keys: {list(checkpoint.keys())}")
    else:
        print(f"❌ Không tìm thấy checkpoint: {ckpt}")

Found checkpoint: D:\BaiduNetdiskDownload\Nudiff-checkpoints\diffusion\monuseg_step1\prop0.1\model130000.pt


C:\Users\ASUS\AppData\Local\Temp\ipykernel_23764\1787546180.py:29: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(ckpt, map_location='cpu')


Keys: ['time_embed.0.weight', 'time_embed.0.bias', 'time_embed.2.weight', 'time_embed.2.bias', 'input_blocks.0.0.weight', 'input_blocks.0.0.bias', 'input_blocks.1.0.in_layers.0.weight', 'input_blocks.1.0.in_layers.0.bias', 'input_blocks.1.0.in_layers.2.weight', 'input_blocks.1.0.in_layers.2.bias', 'input_blocks.1.0.emb_layers.1.weight', 'input_blocks.1.0.emb_layers.1.bias', 'input_blocks.1.0.out_layers.0.weight', 'input_blocks.1.0.out_layers.0.bias', 'input_blocks.1.0.out_layers.3.weight', 'input_blocks.1.0.out_layers.3.bias', 'input_blocks.2.0.in_layers.0.weight', 'input_blocks.2.0.in_layers.0.bias', 'input_blocks.2.0.in_layers.2.weight', 'input_blocks.2.0.in_layers.2.bias', 'input_blocks.2.0.emb_layers.1.weight', 'input_blocks.2.0.emb_layers.1.bias', 'input_blocks.2.0.out_layers.0.weight', 'input_blocks.2.0.out_layers.0.bias', 'input_blocks.2.0.out_layers.3.weight', 'input_blocks.2.0.out_layers.3.bias', 'input_blocks.3.0.in_layers.0.weight', 'input_blocks.3.0.in_layers.0.bias', 'inpu

# 2. Sample

## 2.1 Nuclei structure synthesis

In [5]:
gpu_id = 0
step1 = '130000' # select one checkpoint in step1
prop = 0.1
bs = 1 # 32, 8 thì chậm 
num_samples = 1  # 512
modelpath = step1_ckpt
resdir = f'results/monuseg_mask/prop{prop:.1f}/{step1}_{num_samples}'
assert os.path.exists(modelpath)
print(os.path.exists(resdir))
command = f'CUDA_VISIBLE_DEVICES={gpu_id} OPENAI_LOGDIR={resdir} \
            python scripts/struct_syn/struct_sample.py \
            --model_path {modelpath} --results_path {resdir} \
            --attention_resolutions 32,16,8 --class_cond False --diffusion_steps 1000 --image_size 256 --in_channels 3 \
            --learn_sigma True --noise_schedule linear --num_channels 256 --num_head_channels 64 --num_res_blocks 2 \
            --resblock_updown True --use_fp16 True --use_scale_shift_norm True \
            --batch_size {bs} --num_samples {num_samples}'
print(command)

True
CUDA_VISIBLE_DEVICES=0 OPENAI_LOGDIR=results/monuseg_mask/prop0.1/130000_1             python scripts/struct_syn/struct_sample.py             --model_path D:\BaiduNetdiskDownload\Nudiff-checkpoints\diffusion\monuseg_step1\prop0.1\model130000.pt --results_path results/monuseg_mask/prop0.1/130000_1             --attention_resolutions 32,16,8 --class_cond False --diffusion_steps 1000 --image_size 256 --in_channels 3             --learn_sigma True --noise_schedule linear --num_channels 256 --num_head_channels 64 --num_res_blocks 2             --resblock_updown True --use_fp16 True --use_scale_shift_norm True             --batch_size 1 --num_samples 1


In [8]:
## get instance map from nuclei structures
import scipy.io as sio
from skimage import io
import glob
from tqdm import tqdm
from nudiff.image_syn.utils.post_proc import get_instance_map
import numpy as np

prop = 0.1
datadir = resdir
inst_path = f'{datadir}/instances'
os.makedirs(inst_path, exist_ok=True)
files = sorted(glob.glob(f'{datadir}/samples/*.png'))
names = [os.path.basename(x).split('.')[0] for x in files]  # Sử dụng os.path.basename để tránh lỗi path separator
print(f"Found {len(names)} files: {names}")
masks = [io.imread(x) for x in files]
labels = []
for mask in tqdm(masks):
    label = get_instance_map(mask)
    labels.append(label)
for name, label in zip(names, labels):
    # Lưu dưới dạng .npy thay vì .mat
    save_path = os.path.join(inst_path, f'{name}.npy')
    np.save(save_path, label)
    print(f"Saved {save_path}")

Found 1 files: ['PYey4hyO']


100%|██████████| 1/1 [00:00<00:00, 14.55it/s]

Saved results/monuseg_mask/prop0.1/130000_1/instances\PYey4hyO.npy


## 2.2 Nuclei image synthesis

In [10]:
gpu_id = 0
prop = 0.1
step2 = '150000' # select the checkpoint in step2
s = 2.0
num_samples = 512
modelpath = step2_ckpt
datadir = r'D:\project\Nudiff\results\monuseg_mask\prop0.1\130000_1\samples' # f'results/monuseg_mask/prop{prop:.1f}/{step1}_{num_samples}/samples'
resdir = f'results/monuseg_mask/prop{prop:.1f}/{step1}_{num_samples}/finetune_{step2}_s{s}'
assert os.path.exists(modelpath)
assert os.path.exists(datadir)
print(os.path.exists(resdir))
command = f'CUDA_VISIBLE_DEVICES={gpu_id} OPENAI_LOGDIR={resdir} \
            python scripts/image_syn/sample.py \
            --data_dir {datadir} --model_path {modelpath} --results_path {resdir} --input_mask True \
            --attention_resolutions 32,16,8 --diffusion_steps 1000 --image_size 256 --learn_sigma True \
            --noise_schedule linear --num_channels 256 --num_head_channels 64 --num_res_blocks 2 --resblock_updown True --use_fp16 True \
            --use_scale_shift_norm True --num_classes 2 \
            --class_cond True --no_instance False --batch_size 1 --num_samples {num_samples} --s {s}'
print(command)

False
CUDA_VISIBLE_DEVICES=0 OPENAI_LOGDIR=results/monuseg_mask/prop0.1/130000_512/finetune_150000_s2.0             python scripts/image_syn/sample.py             --data_dir D:\project\Nudiff\results\monuseg_mask\prop0.1\130000_1\samples --model_path D:\BaiduNetdiskDownload\Nudiff-checkpoints\diffusion\monuseg_step2\prop0.1_finetune\model150000.pt --results_path results/monuseg_mask/prop0.1/130000_512/finetune_150000_s2.0 --input_mask True             --attention_resolutions 32,16,8 --diffusion_steps 1000 --image_size 256 --learn_sigma True             --noise_schedule linear --num_channels 256 --num_head_channels 64 --num_res_blocks 2 --resblock_updown True --use_fp16 True             --use_scale_shift_norm True --num_classes 2             --class_cond True --no_instance False --batch_size 1 --num_samples 512 --s 2.0
